In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from TCGA_files import *
from gtex import get_gtex_tissue
import seaborn as sns

In [ ]:
l = 3
#setup = '5000doc_0.25occ_fpkmweight_thr'
#setup='overlap_5000doc_0.25occ_fpkmweight_thr'
#setup = 'oversigma_10tissue'
#label = 'disease_type'
label='primary_site'
#directory = '/Users/filippo/Developer/tesi/results/hSBM/%s/topsbm'%(setup)
#directory = '/Users/filippo/Developer/tesi/gtex/hsbm/%s'%setup
directory='/Users/filippo/Developer/tesi/merged/hsbm'

In [ ]:
df = pd.read_csv("%s/topsbm/topsbm_level_%d_topic-dist.csv"%(directory,l))
df.head()

In [ ]:
topic = 15

In [ ]:
searchdf = df.sort_values(by="Topic %d"%topic, ascending=False).loc[:,['doc','Topic %d'%topic]]
searchdf.head()

In [ ]:
#datatotest = queryFiles([f[0]+'.FPKM.txt.gz' for f in searchdf.values[:30] if f[1]>0.1])
#datatotest = queryFiles([f[0] for f in searchdf.values[:10]])
datatotest = pd.DataFrame(columns=['primary_site','secondary_site'])
for file in [f[0] for f in searchdf.values[:10]]:
    datatotest=datatotest.append(get_gtex_tissue(file))
datatotest

In [ ]:
makeTopicPie(datatotest, l, ['primary_site','secondary_site'])

In [ ]:
df_file=pd.read_csv("files.dat", index_col=[0])
samples = []
for sample in df['doc']:
    if 'Lung' in get_gtex_tissue(sample)['primary_site']:
        samples.append(sample)

In [ ]:
tissuedf = df[df['doc'].isin(samples)].drop('i_doc', axis=1)
tissuedf.mean(axis=0).sort_values(ascending=False)

## topic distr

In [ ]:
def topic_distr_sample(doc, ax=None):
    if ax == None:
        fig = plt.figure()
        ax = fig.subplots()
    ax.set_title("Topic distribution: %s"%doc)
    labels=[l if df[df['doc']==doc].loc[:,l].values[0]>=0.05 else '' for l in df.columns[2:]]
    patches, texts, autotexts = ax.pie(df[df['doc']==doc].values[0][2:], labels=labels, autopct=lambda p: '%.1f%s'%(p,'%') if p >= 5 else '', textprops={'fontsize':20, 'color':'white', 'wrap':True})
    for t in texts:
                t.set_fontsize(18)
                t.set_wrap(True)
                t.set_color('black')
    plt.show()
    
def topic_distr_isample(idoc, ax=None):
    topic_distr_sample(df[df['i_doc']==idoc]['doc'].values[0],ax)

In [ ]:
for idoc in searchdf.index.values[:5]:
    fig=plt.figure()
    ax=fig.subplots()
    topic_distr_isample(idoc,ax)
    plt.show()
    fig.savefig("topic_distr_%d.png"%idoc)

In [ ]:
for idoc in np.random.randint(len(df.index), size=10):
    topic_distr_isample(idoc)

## Topic distr

### kl

In [ ]:
l=3

In [ ]:
df_kl = pd.read_csv("%s/topsbm/topic-kl_%d.csv"%(directory,l), header=None)
df_kl.columns = ['first', 'second', 'kl']
df_kl.head()

In [ ]:
df_cluster = pd.read_csv("%s/topsbm/topsbm_level_%d_clusters.csv"%(directory,l))
df_topics = pd.read_csv("%s/topsbm/topsbm_level_%d_topic-dist.csv"%(directory,l)).loc[:,df.columns[2:]]
df_files = pd.read_csv("%s/files.dat"%(directory), index_col=[0], header=0)

In [ ]:
bins = np.linspace(-0.025,1.025,40)

In [ ]:
sites = df_files['primary_site'].unique()
df_tissue_kl = pd.DataFrame(index=sites, columns=sites, dtype=float)

In [ ]:
for tissue_row in sites:
    cluster_row = df_files[df_files['primary_site']==tissue_row].index.values
    for tissue_column in sites:
        print(tissue_row, tissue_column)
        cluster_column = df_files[df_files['primary_site']==tissue_column].index.values
        datarc = df_kl[(df_kl['first'].isin(cluster_row) & df_kl['second'].isin(cluster_column))]['kl'].values
        datacr = df_kl[(df_kl['first'].isin(cluster_column) & df_kl['second'].isin(cluster_row))]['kl'].values
        df_tissue_kl.at[tissue_row,tissue_column]=(np.average(np.concatenate((datarc,datacr))))

In [ ]:
h = sns.clustermap(df_tissue_kl.dropna(axis=0,how='all').dropna(axis=1, how='any'), cmap=sns.diverging_palette(15,250, n=15), metric='euclidean')
dn = h.dendrogram_col.dendrogram
h.fig.savefig("%s/topic_distr_kl_map.pdf"%directory)

In [ ]:
import scipy.cluster.hierarchy as shc
fig = plt.figure(figsize=(12,8))
ax = fig.subplots()
ax.set_xlabel("kl correlation", fontsize=16)
dend = shc.dendrogram(h.dendrogram_col.linkage, labels=df_tissue_kl.columns, orientation='right', distance_sort='discending', ax=ax)
fig.savefig("%s/topic_dist_dendogram_level(%d).pdf"%(directory,l))

### Topic maps

In [ ]:
import seaborn as sns
from gtex import get_gtex_tissue
from TCGA_files import get_tcga_tissue
from hsbmpy import get_file, get_tissue_style

In [ ]:
df_topics = pd.read_csv("%s/topsbm/topsbm_level_%d_topic-dist.csv"%(directory,l))
df_files = pd.read_csv("%s/files.dat"%directory, index_col=0)
df_topics.set_index('doc', inplace=True)
df_topics.insert(0,'tissue','')
df_topics.drop('i_doc', axis=1, inplace=True)
for sample in df_topics.index.values:
    df_topics.at[sample,'tissue']=("%s"%(get_file(sample,df_files)['primary_site']))

In [ ]:
df_cmap = df_topics.sort_values(by='tissue').set_index('tissue').transpose()
#create a color palette with the same number of colors as unique values in the Source column
network_pal = sns.color_palette('husl',n_colors=len(df_cmap.columns))

#Create a dictionary where the key is the category and the values are the
#colors from the palette we just created
network_lut = dict(zip(df_cmap.columns, network_pal))
network_col = df_cmap.columns.map(network_lut)

fig = plt.figure()
cm = sns.clustermap(df_cmap, row_cluster=False, col_cluster=False, metric='euclidean', vmin=0, cmap='RdYlBu_r', col_colors=network_col)
cm.fig.savefig("%s/MAP_level%d.png"%(directory,l))

In [ ]:
df_topics = pd.read_csv("%s/topsbm/topsbm_level_%d_topic-dist.csv"%(directory,l))
df_topics.drop('i_doc', axis=1, inplace=True)
df_topics.set_index('doc', inplace=True)
for sample in df_topics.index.values:
    df_topics.at[sample,'tissue']="%s"%(get_file(sample,df_files)['primary_site'])

In [ ]:
fig = plt.figure(figsize=(25,10))
for tissue in df_topics['tissue'].unique():
    print(tissue)
    marker, c, ls = get_tissue_style(tissue)
    try:
        plt.plot(df_topics[df_topics['tissue']==tissue].loc[:,df_topics.columns[30:65]].mean(axis=0), ls=ls,marker=marker, lw=2, ms=10, label=tissue, c=network_lut[df_files[df_files['primary_site']==tissue]['primary_site'][0]])
    except:
        pass
plt.xticks(rotation=90)
plt.ylabel("$P(topic | tissue)$", fontsize=18)
plt.legend(fontsize=18)
plt.show()
fig.savefig("%s/lifeplot.pdf"%directory)

In [ ]:
expressedtopics = df_topics.drop('tissue', axis=1).mean(axis=0).sort_values(ascending=False).index

In [ ]:
fig=plt.figure()
for thr in [0, 0.005, 0.01]:
    plt.hist(df_topics.drop('tissue', axis=1).transpose().apply(lambda x: len(x[x>thr])/float(len(x)), axis=1), histtype='step', density=True, range=(-0.05,1.05), bins=10, lw=2, label="threshold: %.3f"%thr)
plt.legend()
fig.savefig("%s/distr_topicU.pdf"%directory)

In [ ]:
topic_table = df_topics.drop('tissue', axis=1).transpose()
topic_table.insert(0,'occ', df_topics.drop('tissue', axis=1).transpose().apply(lambda x: len(x[x>0.002])/float(len(x)), axis=1).values)

In [ ]:
low_occ_topics = topic_table[topic_table['occ']>0.5].index.values

In [ ]:
import findspark
findspark.init()
import pyspark as spark
import tensorflow as tf
from pyspark.sql.functions import udf,col
from pyspark.sql.types import StringType
sc = spark.SparkContext()
sql = spark.SQLContext(sc)

In [ ]:
df_files_pd = pd.read_csv("%s/files.dat"%directory, index_col=0)
df_topics = sql.read.option("header","true").csv("%s/topsbm/topsbm_level_%d_topic-dist.csv"%(directory,l))
df_files = sql.read.option("header","true").csv("%s/files.dat"%(directory))
df_files.registerTempTable("file")
df_topics = df_topics.withColumn('status', udf(lambda x: 'healthy' if 'GTEX' in x else 'tumour', StringType())(col('doc')))
df_topics = df_topics.withColumn('tissue', udf(lambda x: get_file(x, df_files_pd)['primary_site'], StringType())(col('doc')))
df_topics = df_topics.withColumn('second_tissue', udf(lambda x: get_file(x, df_files_pd)['secondary_site'], StringType())(col('doc')))
df_topics.registerTempTable("topic")

In [ ]:
dftf_h = tf.convert_to_tensor(sql.sql("SELECT * FROM topic WHERE status='healthy'").toPandas().drop(['doc','i_doc', 'status', 'tissue','statustissue'], axis=1).astype(float).values)
dftf_d = tf.convert_to_tensor(sql.sql("SELECT * FROM topic WHERE status='tumour'").toPandas().drop(['doc','i_doc', 'status', 'tissue','statustissue'], axis=1).astype(float).values)

In [ ]:
with tf.Session() as sess:
    results = sess.run([tf.math.reduce_mean(dftf_h,0),tf.math.reduce_mean(dftf_d,0)])

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(results[0], marker='o', lw=0.1)
plt.plot(results[1],marker='x', lw=0.1)
plt.show()

In [ ]:
df_all = tf.convert_to_tensor(sql.sql("SELECT * FROM topic").toPandas().drop(['i_doc', 'doc', 'status', 'tissue','statustissue'], axis=1).astype(float))

In [ ]:
#normed_df = tf.divide(tf.subtract(df_all,tf.reduce_mean(df_all,0)),tf.math.reduce_mean(df_all, 0))
normed_df = tf.divide(tf.abs(tf.subtract(df_all,tf.reduce_mean(df_all,0))),tf.sqrt(tf.math.reduce_variance(df_all, 0)))
#normed_df = tf.divide(tf.math.divide(df_all,tf.reduce_mean(df_all,0)), tf.cast(tf.shape(df_all)[0], tf.float64))
#normed_df = tf.math.multiply(df_all,tf.reduce_mean(df_all,0))

In [ ]:
with tf.Session() as sess:
    result = sess.run([normed_df])

In [ ]:
l_topics_i = [int(t[6:])-1 for t in low_occ_topics]
#l_topics_i = np.random.randint(0,100,10)
fig=plt.figure(figsize=(30,15))
label='second_tissue'
for tissue in df_topics.select(label).distinct().toPandas().values.T[0][:]:
#for tissue in ['breast-gtex', 'breast-tcga', 'colon-gtex', 'colon-tcga','thyroid-gtex','thyroid-tcga']:
    print(tissue)
    marker, c, ls = get_tissue_style(tissue)
    i_docs = sql.sql("SELECT i_doc, %s FROM topic WHERE %s='%s'"%(label, label,tissue)).select('i_doc').toPandas().astype(int).values.T[0]
    plt.plot(np.mean(result[0][i_docs],axis=0)[l_topics_i], marker=marker, lw=0.1, ls=ls, label=tissue, ms=18, c=c)
plt.legend(fontsize=16, ncol=4)
tf.divide(tf.abs(tf.subtract(df_all,tf.reduce_mean(df_all,0))),tf.sqrt(tf.math.reduce_variance(df_all, 0)))
plt.ylabel("$\\frac{\\left|P(topic | sample) - mean\\right|}{\sigma}$", fontsize=18)
plt.xticks(ticks=np.linspace(0,len(l_topics_i)-1,num=len(l_topics_i)), labels=low_occ_topics, rotation=90)
plt.show()
fig.savefig("%s/lifeplot_normalised_%s.pdf"%(directory,label))

In [ ]:
tf.reset_default_graph()
sc.stop()

In [ ]:
import importlib, hsbmpy
importlib.reload(hsbmpy)
from hsbmpy import get_tissue_style